In [39]:
%pip install zkstats

import os
from zkstats.core import (
    prover_gen_settings,
    prover_gen_proof,
    verifier_setup,
    verifier_verify,
)
from zkstats.computation import computation_to_model

cwd = os.getcwd()

# FIXME: fill this in with the path to your data
data_path = f"{cwd}/data.json"

# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"

pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [40]:
# This is just a dummy computation. Replace it with user's computation
#
# import torch
# from zkstats.computation import State
#
# def computation(state: State, x: list[torch.Tensor]):
#     x_0 = x[0]
#     out_0 = state.median(x_0)
#     out_1 = state.median(x_0)
#     return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

Generate settings and setup with user's computation.

In [41]:
_, model = computation_to_model(computation)
prover_gen_settings(
    [data_path],
    comb_data_path,
    model,
    model_onnx_path,
    "default",
    "resources",
    settings_path,
)
verifier_setup(model_onnx_path, compiled_model_path, settings_path, vk_path, pk_path)

Generate proof with your data and user's computation.

In [42]:
prover_gen_proof(
    model_onnx_path,
    comb_data_path,
    witness_path,
    compiled_model_path,
    settings_path,
    proof_path,
    pk_path,
)


Verify the proof to ensure it is correct

In [43]:
verifier_verify(proof_path, settings_path, vk_path)

Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [44]:
model_onnx_path, settings_path, proof_path, vk_path
print("Model onnx:\t\t", model_onnx_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", vk_path)